In [5]:
import requests

/Users/macbookpro/Desktop/AI SIBERIA/Computer Vision /.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
url = "https://www.dropbox.com/s/5jh4hpuk2gcxaaq/all.zip"
local_filename = "/Users/macbookpro/Desktop/AI SIBERIA/Computer Vision /Chapter-4/data/all.zip"

response = requests.get(url)
with open(local_filename, 'wb') as f:
    f.write(response.content)